## Continuous Reprocessing Examples

Two isotopes A and B for some element. Can vary decay constants, production rates, and reprocessing constants for each isotope.

Bateman equation for each isotope given below as well as solution.

$\frac{dN}{dt} = -\lambda_d N - \lambda_r N + prod$

$N(t) = N_0 e^{(-\lambda_{d} + \lambda_{r}) t} + \frac{prod}{\lambda_d + \lambda_r} \left( 1 - e^{(-\lambda_{d} + \lambda_{r}) t}   \right)$


#### Explanation of physicality

In reality, when removing part of an element, we expect both isotopes to be removed proportionally. This means that the ratio of isotopes should remain constant while we perform reprocessing.

This is what we see during a simple case (stable isotopes, no production). However, once production of isotopes gets involved, things become more complicated.

If production is a constant value and is the same for both isotopes, it will drive the ratio to be 1 (A/B = 1).

#### Continuous vs Batchwise

The differences become present when production appears. It seems that decay and reprocessing work well on continuous and batch, but production causes issues.

Likely, continuous reprocesses produced isotopes as they are generated while batch does not affect them until the final reprocess step.

This means that a high-production and rapid removal isotope would be poorly modeled using batchwise reprocessing unless very small time steps are used.




In [34]:
def continuous(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times):
    print('Continuous Reprocessing')
    tabl = []
    tabl.append(['Time', 'N(t) A', 'N(t) B', 'Net', 'A/B'])
    for t in times:
        if (reproc_a + decay_a) > 0:
            n_a = init_a * np.exp(-t * (reproc_a + decay_a)) + prod_a/(reproc_a + decay_a) * (1 - np.exp(-t * (reproc_a + decay_a)))
        else:
            n_a = init_a + prod_a * t
        if (reproc_b + decay_b) > 0:
            n_b = init_b * np.exp(-t * (reproc_b + decay_b)) + prod_b/(reproc_b + decay_b) * (1 - np.exp(-t * (reproc_b + decay_b)))
        else:
            n_b = init_b + prod_b * t
        net = n_a + n_b
        tabl.append([t, n_a, n_b, net, n_a/n_b])
    print(tabulate(tabl, headers='firstrow', tablefmt='fancy_grid'))
    return tabl

In [33]:
def batch(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times):
    print('Batchwise Reprocessing')
    tabl = []
    tabl.append(['Time', 'N(t) A', 'N(t) B', 'Net', 'A/B'])
    step_a = init_a
    step_b = init_b
    X_a = 1 - np.exp(-reproc_a)
    X_b = 1 - np.exp(-reproc_b)
    reproc_a = 0
    reproc_b = 0
    for index_t, t in enumerate(times):
        if index_t == 0:
            n_a = init_a
            n_b = init_b
            net = n_a + n_b
            tabl.append([t, n_a, n_b, net, n_a/n_b])
            continue
        dt = t - times[index_t - 1]
        if (reproc_a + decay_a) > 0:
            n_a = step_a * np.exp(-dt * (reproc_a + decay_a)) + prod_a/(reproc_a + decay_a) * (1 - np.exp(-dt * (reproc_a + decay_a)))
        else:
            n_a = step_a + prod_a * dt
        if (reproc_b + decay_b) > 0:
            n_b = step_b * np.exp(-dt * (reproc_b + decay_b)) + prod_b/(reproc_b + decay_b) * (1 - np.exp(-dt * (reproc_b + decay_b)))
        else:
            n_b = step_b + prod_b * dt
        if t != 0:
            n_a = (1-X_a) * n_a
            n_b = (1-X_b) * n_b
        net = n_a + n_b
        tabl.append([t, n_a, n_b, net, n_a/n_b])
        step_a = n_a
        step_b = n_b
    print(tabulate(tabl, headers='firstrow', tablefmt='fancy_grid'))
    return tabl

### Stable isotopes, no production, same reprocessing constants

$\frac{A}{B}$ ratio constant, physical result. First is continuous, second is batchwise.

In [36]:
import numpy as np
from tabulate import tabulate
initial = 100
init_a = 0.9 * initial
init_b = 0.1 * initial
X = 0.1
reproc_a = np.log(1 / (1 - X))
reproc_b = np.log(1 / (1 - X))
decay_a = 0
decay_b = 0
prod_a = 0
prod_b = 0
times = np.arange(0, 4)


cont_table = continuous(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)
batch_table = batch(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)
    


Continuous Reprocessing
╒════════╤══════════╤══════════╤═══════╤═══════╕
│   Time │   N(t) A │   N(t) B │   Net │   A/B │
╞════════╪══════════╪══════════╪═══════╪═══════╡
│      0 │    90    │    10    │ 100   │     9 │
├────────┼──────────┼──────────┼───────┼───────┤
│      1 │    81    │     9    │  90   │     9 │
├────────┼──────────┼──────────┼───────┼───────┤
│      2 │    72.9  │     8.1  │  81   │     9 │
├────────┼──────────┼──────────┼───────┼───────┤
│      3 │    65.61 │     7.29 │  72.9 │     9 │
╘════════╧══════════╧══════════╧═══════╧═══════╛
Batchwise Reprocessing
╒════════╤══════════╤══════════╤═══════╤═══════╕
│   Time │   N(t) A │   N(t) B │   Net │   A/B │
╞════════╪══════════╪══════════╪═══════╪═══════╡
│      0 │    90    │    10    │ 100   │     9 │
├────────┼──────────┼──────────┼───────┼───────┤
│      1 │    81    │     9    │  90   │     9 │
├────────┼──────────┼──────────┼───────┼───────┤
│      2 │    72.9  │     8.1  │  81   │     9 │
├────────┼──────────┼─

### Stable isotopes, no production, different reprocessing constants

$\frac{A}{B}$ ratio changes due to this, non-physical result. First continuous, second batch.

In [37]:
import numpy as np
from tabulate import tabulate
initial = 100
init_a = 0.9 * initial
init_b = 0.1 * initial
X = 0.1
reproc_a = np.log(1 / (1 - X))
reproc_b = np.log(1 / (1 - X*2))
decay_a = 0
decay_b = 0
prod_a = 0
prod_b = 0
times = np.arange(0, 4)


cont_table = continuous(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)
batch_table = batch(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)

Continuous Reprocessing
╒════════╤══════════╤══════════╤════════╤═════════╕
│   Time │   N(t) A │   N(t) B │    Net │     A/B │
╞════════╪══════════╪══════════╪════════╪═════════╡
│      0 │    90    │    10    │ 100    │  9      │
├────────┼──────────┼──────────┼────────┼─────────┤
│      1 │    81    │     8    │  89    │ 10.125  │
├────────┼──────────┼──────────┼────────┼─────────┤
│      2 │    72.9  │     6.4  │  79.3  │ 11.3906 │
├────────┼──────────┼──────────┼────────┼─────────┤
│      3 │    65.61 │     5.12 │  70.73 │ 12.8145 │
╘════════╧══════════╧══════════╧════════╧═════════╛
Batchwise Reprocessing
╒════════╤══════════╤══════════╤════════╤═════════╕
│   Time │   N(t) A │   N(t) B │    Net │     A/B │
╞════════╪══════════╪══════════╪════════╪═════════╡
│      0 │    90    │    10    │ 100    │  9      │
├────────┼──────────┼──────────┼────────┼─────────┤
│      1 │    81    │     8    │  89    │ 10.125  │
├────────┼──────────┼──────────┼────────┼─────────┤
│      2 │    72.

### Equally unstable isotopes, no production, same reprocessing constants

Both decay at same rate, $\frac{A}{B}$ ratio unchanged.

In [38]:
import numpy as np
from tabulate import tabulate
initial = 100
init_a = 0.9 * initial
init_b = 0.1 * initial
X = 0.1
reproc_a = np.log(1 / (1 - X))
reproc_b = np.log(1 / (1 - X))
decay_a = 1
decay_b = 1
prod_a = 0
prod_b = 0
times = np.arange(0, 4)


cont_table = continuous(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)
batch_table = batch(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)

Continuous Reprocessing
╒════════╤══════════╤═══════════╤═══════════╤═══════╕
│   Time │   N(t) A │    N(t) B │       Net │   A/B │
╞════════╪══════════╪═══════════╪═══════════╪═══════╡
│      0 │ 90       │ 10        │ 100       │     9 │
├────────┼──────────┼───────────┼───────────┼───────┤
│      1 │ 29.7982  │  3.31091  │  33.1091  │     9 │
├────────┼──────────┼───────────┼───────────┼───────┤
│      2 │  9.86594 │  1.09622  │  10.9622  │     9 │
├────────┼──────────┼───────────┼───────────┼───────┤
│      3 │  3.26653 │  0.362948 │   3.62948 │     9 │
╘════════╧══════════╧═══════════╧═══════════╧═══════╛
Batchwise Reprocessing
╒════════╤══════════╤═══════════╤═══════════╤═══════╕
│   Time │   N(t) A │    N(t) B │       Net │   A/B │
╞════════╪══════════╪═══════════╪═══════════╪═══════╡
│      0 │ 90       │ 10        │ 100       │     9 │
├────────┼──────────┼───────────┼───────────┼───────┤
│      1 │ 29.7982  │  3.31091  │  33.1091  │     9 │
├────────┼──────────┼───────────┼──

### Unstable isotopes, no production, same reprocessing constants

Decay at different rate, $\frac{A}{B}$ ratio changes. Physical since ratio changes same amount as without reprocessing.

In [40]:
import numpy as np
from tabulate import tabulate
initial = 100
init_a = 0.9 * initial
init_b = 0.1 * initial
X = 0.1
reproc_a = np.log(1 / (1 - X))
reproc_b = np.log(1 / (1 - X))
decay_a = 1
decay_b = 0.5
prod_a = 0
prod_b = 0
times = np.arange(0, 4)


cont_table = continuous(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)
batch_table = batch(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)

print('-'*54)
print('Reprocessing disabled')
print('-'*54)

reproc_a = 0
reproc_b = 0
cont_table = continuous(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)
batch_table = batch(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)


Continuous Reprocessing
╒════════╤══════════╤══════════╤═══════════╤═════════╕
│   Time │   N(t) A │   N(t) B │       Net │     A/B │
╞════════╪══════════╪══════════╪═══════════╪═════════╡
│      0 │ 90       │ 10       │ 100       │ 9       │
├────────┼──────────┼──────────┼───────────┼─────────┤
│      1 │ 29.7982  │  5.45878 │  35.257   │ 5.45878 │
├────────┼──────────┼──────────┼───────────┼─────────┤
│      2 │  9.86594 │  2.97982 │  12.8458  │ 3.31091 │
├────────┼──────────┼──────────┼───────────┼─────────┤
│      3 │  3.26653 │  1.62662 │   4.89315 │ 2.00817 │
╘════════╧══════════╧══════════╧═══════════╧═════════╛
Batchwise Reprocessing
╒════════╤══════════╤══════════╤═══════════╤═════════╕
│   Time │   N(t) A │   N(t) B │       Net │     A/B │
╞════════╪══════════╪══════════╪═══════════╪═════════╡
│      0 │ 90       │ 10       │ 100       │ 9       │
├────────┼──────────┼──────────┼───────────┼─────────┤
│      1 │ 29.7982  │  5.45878 │  35.257   │ 5.45878 │
├────────┼────────

### Unstable isotopes, same production, same reprocessing constants

First full, second is without reprocessing, third is without production

Produce at same rate, $\frac{A}{B}$ ratio changes. Ratio changes with removal of reprocessing. Check against batchwise instead.

In [42]:
import numpy as np
from tabulate import tabulate
initial = 100
init_a = 0.9 * initial
init_b = 0.1 * initial
X = 0.1
reproc_a = np.log(1 / (1 - X))
reproc_b = np.log(1 / (1 - X))
decay_a = 0.1
decay_b = 0.1
prod_a = 5
prod_b = 3
times = np.arange(0, 4)


cont_table = continuous(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)
batch_table = batch(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)
    

print('-'*54)
print('Reprocessing disabled')
print('-'*54)

reproc_a = 0
reproc_b = 0
cont_table = continuous(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)
batch_table = batch(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)

print('-'*54)
print('Production disabled')
print('-'*54)

reproc_a = np.log(1 / (1 - X))
reproc_b = np.log(1 / (1 - X))
prod_a = 0
prod_b = 0
cont_table = continuous(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)
batch_table = batch(init_a, init_b, reproc_a, reproc_b, decay_a, decay_b, prod_a, prod_b, times)


Continuous Reprocessing
╒════════╤══════════╤══════════╤══════════╤═════════╕
│   Time │   N(t) A │   N(t) B │      Net │     A/B │
╞════════╪══════════╪══════════╪══════════╪═════════╡
│      0 │  90      │  10      │ 100      │ 9       │
├────────┼──────────┼──────────┼──────────┼─────────┤
│      1 │  77.8118 │  10.8555 │  88.6674 │ 7.16794 │
├────────┼──────────┼──────────┼──────────┼─────────┤
│      2 │  67.8864 │  11.5523 │  79.4386 │ 5.87646 │
├────────┼──────────┼──────────┼──────────┼─────────┤
│      3 │  59.8035 │  12.1196 │  71.9232 │ 4.93444 │
╘════════╧══════════╧══════════╧══════════╧═════════╛
Batchwise Reprocessing
╒════════╤══════════╤══════════╤══════════╤═════════╕
│   Time │   N(t) A │   N(t) B │      Net │     A/B │
╞════════╪══════════╪══════════╪══════════╪═════════╡
│      0 │  90      │  10      │ 100      │ 9       │
├────────┼──────────┼──────────┼──────────┼─────────┤
│      1 │  77.5741 │  10.7129 │  88.2871 │ 7.24117 │
├────────┼──────────┼──────────┼───